## Introduction: Business Problem
#### I want to find a location for my Yoga Studio in Toronto. For this purpose I  I'm going to study the different neighbourhoods, if they already have a Yoga Studio, and the characteristics of the neighbourhood. 

# Data
### Number of venues and their type and location in every neighborhood will be obtained using Foursquare API
### coordinate of Toronto center will be obtained using Google Maps API geocoding
### url - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M for extracting data

# Methodology
### First I am going to get the neighbourhoods of Toronto. Once I got them, I'm going to retrieve the information of venues and comertial activity in the area using four square.
### Then I am going to filter the neighbourhoods without Yoga Studios as we don't want competence in the neighbourhood.
### Finally, I am going to inspect the possible neighbourhoods to determine the most suitable.

In [1]:
!pip install bs4
!pip install geopy
!conda update pandas # Update pandas to resolve environment mismatch
!conda install beautifulsoup4 --yes
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library



print('Libraries imported.')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Solving environment: - 

Updating pandas is constricted by 

ibm-wsrt-py37omain-main -> requ

In [30]:
#Beautiful Soup retrieve html data from wikipedia
from bs4 import BeautifulSoup
Url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
TorontoData  = requests.get(Url).text
soup = BeautifulSoup(TorontoData, 'html5lib')

#Scraping data from wokipedia and convert into df

table_contents=[]
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.shape
df.sort_values(by=['PostalCode'])

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
32,M1J,Scarborough,Scarborough Village
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
58,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [31]:
#Retreive Lat & Long data using Geospatial_Coordinates.csv
df_geo = pd.read_csv (r'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
print (df_geo)


    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750072 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24          M2R  43.782736 -79.442259
25          

In [32]:
#Merge data 
df_geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
neighborhoods = pd.merge(df, df_geo, on='PostalCode', how='inner')
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [33]:
#Explore and cluster the neighborhoods in Toronto

neighborhoods['Borough'].value_counts()

North York                24
Scarborough               17
Downtown Toronto          17
Etobicoke                 11
Central Toronto            9
West Toronto               6
York                       5
East Toronto               4
East York                  4
Mississauga                1
East York/East Toronto     1
Downtown Toronto Stn A     1
Queen's Park               1
East Toronto Business      1
Etobicoke Northwest        1
Name: Borough, dtype: int64

In [34]:
#Identify Longitude & Latitude of Toronto
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode('Toronto')
TO_latitude = location.latitude
TO_longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(TO_latitude, TO_longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [35]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[TO_latitude, TO_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [36]:
#Foursquare
#setting the radius and limit 
radius=500
LIMIT=100

In [37]:
CLIENT_ID = 'EJYD2L52IY03RRLHYVXDVWSYS2MHQVFW4FDI0JJZ5CG4MNIF' # your Foursquare ID
CLIENT_SECRET = 'MIA1POB2TUYW3QDLO42I4BP2SBXZ5XXZW04N4B50VB5QPEXU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('EJYD2L52IY03RRLHYVXDVWSYS2MHQVFW4FDI0JJZ5CG4MNIF: ' + CLIENT_ID)
print('MIA1POB2TUYW3QDLO42I4BP2SBXZ5XXZW04N4B50VB5QPEXU:' + CLIENT_SECRET)

Your credentails:
EJYD2L52IY03RRLHYVXDVWSYS2MHQVFW4FDI0JJZ5CG4MNIF: EJYD2L52IY03RRLHYVXDVWSYS2MHQVFW4FDI0JJZ5CG4MNIF
MIA1POB2TUYW3QDLO42I4BP2SBXZ5XXZW04N4B50VB5QPEXU:MIA1POB2TUYW3QDLO42I4BP2SBXZ5XXZW04N4B50VB5QPEXU


In [10]:
#we want to obtain all the venues, its category, its Neighbourhood and location 

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
venues_toronto = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [39]:
venues_toronto.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [14]:
#We list all the venue category to find the name of the one that interest us: Yoga Studio

In [40]:
venues_toronto['Venue Category'].unique()

array(['Park', 'Fast Food Restaurant', 'Food & Drink Shop',
       'Hockey Arena', 'Coffee Shop', 'Portuguese Restaurant',
       'French Restaurant', 'Bakery', 'Distribution Center', 'Restaurant',
       'Pub', 'Spa', 'Historic Site', 'Gym / Fitness Center',
       'Chocolate Shop', 'Performing Arts Venue', 'Farmers Market',
       'Breakfast Spot', 'Dessert Shop', 'Mexican Restaurant',
       'Yoga Studio', 'Café', 'Theater', 'Event Space', 'Shoe Store',
       'Art Gallery', 'Electronics Store', 'Beer Store', 'Bank', 'Hotel',
       'Wine Shop', 'Antique Shop', 'Boutique', 'Furniture / Home Store',
       'Vietnamese Restaurant', 'Clothing Store', 'Accessories Store',
       'Gift Shop', 'Italian Restaurant', 'Beer Bar',
       'Persian Restaurant', 'Creperie', 'Hobby Shop', 'Sushi Restaurant',
       'Diner', 'Burrito Place', 'Fried Chicken Joint',
       'Japanese Restaurant', 'Smoothie Shop', 'Sandwich Place', 'Gym',
       'College Auditorium', 'Bar', 'Music Venue', 'Caribbean R

In [41]:
Yoga = venues_toronto[venues_toronto['Venue Category'] == 'Yoga Studio']

In [42]:
Yoga

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
31,"Regent Park, Harbourfront",43.654260,-79.360636,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
69,Ontario Provincial Government,43.662301,-79.389494,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
493,Central Bay Street,43.657952,-79.387383,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
591,Thorncliffe Park,43.705369,-79.349372,Bikram Yoga East York,43.705450,-79.351448,Yoga Studio
928,"Little Portugal, Trinity",43.647927,-79.419750,YogaSpace,43.647607,-79.420133,Yoga Studio
974,"The Danforth West, Riverdale",43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio
1227,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Bikram Yoga Centre,43.649214,-79.375229,Yoga Studio
1299,Studio District,43.659526,-79.340923,Spirit Loft Yoga,43.663548,-79.341333,Yoga Studio
1462,North Toronto West,43.715383,-79.405678,Barreworks,43.714070,-79.400109,Yoga Studio
1576,"University of Toronto, Harbord",43.662696,-79.400049,Sivananda Yoga Centre,43.662754,-79.402951,Yoga Studio


In [43]:
#number of Yoga Studios in the city
Yoga.shape

(14, 7)

In [44]:
#Plot the Yoga studios in a map
map_toronto2 = folium.Map(location=[TO_latitude, TO_longitude], zoom_start=10)
for lat1, lng2, Venue in zip(Yoga['Neighbourhood Latitude'], Yoga['Neighbourhood Longitude'], Yoga['Venue']):
    label2 = '{}'.format(Venue)
    label2 = folium.Popup(label2, parse_html=True)
    folium.CircleMarker(
        [lat1, lng2],
        radius=5,
        popup=label2,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

In [51]:
#find neighgbourhoods without any Yoga Studio
neighborhoods.rename(columns={'Neighborhood':'Neighbourhood'},inplace=True)
neighborhoodsYoga=pd.merge(neighborhoods[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']],Yoga[['Neighbourhood','Venue']],on=['Neighbourhood'],how='left')
neighborhoodsYoga.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,The Yoga Lounge
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,NaN
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,The Yoga Sanctuary


In [52]:
neighborhoodsYoga.shape

(104, 6)

In [54]:
neighborhoodsWithoutYoga=neighborhoodsYoga.drop(neighborhoodsYoga[neighborhoodsYoga.Venue.notnull()].index)
neighborhoodsWithoutYoga.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,NaN
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,NaN


In [55]:
neighborhoodsWithoutYoga.shape

(90, 6)

### We got the neighbourhoods without yoga studio.
### Now we want to find if those neighbourhoods without yoga studio, have a vegetarian/vegan restaurant as the public could be potentially the same

In [56]:
VegRest = venues_toronto[venues_toronto['Venue Category'] == 'Vegetarian / Vegan Restaurant']
VegRest.shape

(18, 7)

In [57]:
VegRest.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
276,St. James Town,43.651494,-79.375418,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
370,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
489,Central Bay Street,43.657952,-79.387383,Vegetarian Haven,43.656016,-79.392758,Vegetarian / Vegan Restaurant
617,"Richmond, Adelaide, King",43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
633,"Richmond, Adelaide, King",43.650571,-79.384568,Planta Queen,43.650622,-79.388154,Vegetarian / Vegan Restaurant


In [58]:
#get how many vegetarian restaurant has each Neighbourhood without Yoga Studio
VegRest.rename(columns={'Venue':'VegRest'},inplace=True)
neighborhoodsWithoutYogaWithVeg=pd.merge(neighborhoodsWithoutYoga[['PostalCode','Borough','Neighbourhood','Latitude','Longitude','Venue']],VegRest[['Neighbourhood','VegRest']],on=['Neighbourhood'],how='left')
neighborhoodsWithoutYogaWithVeg

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Venue,VegRest
0,M3A,North York,Parkwoods,43.753259,-79.329656,NaN,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,NaN,NaN
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,NaN,NaN
3,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN
4,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,NaN,NaN
5,M3B,North York,Don Mills North,43.745906,-79.352188,NaN,NaN
6,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,NaN,NaN
7,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,NaN,NaN
8,M6B,North York,Glencairn,43.709577,-79.445073,NaN,NaN
9,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,NaN,NaN


In [59]:
neighborhoodsWithoutYogaWithVeg.shape

(95, 7)

In [65]:
neighborhoodsWithoutYogaWithVeg_Group=neighborhoodsWithoutYogaWithVeg.groupby(['Neighbourhood']).count()
neighborhoodsWithoutYogaWithVeg_Group

,PostalCode,Borough,Latitude,Longitude,Venue,VegRest
Neighbourhood,,,,,,
Agincourt,1,1,1,1,0,0
"Alderwood, Long Branch",1,1,1,1,0,0
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,0,0
Bayview Village,1,1,1,1,0,0
"Bedford Park, Lawrence Manor East",1,1,1,1,0,0
Berczy Park,1,1,1,1,0,1
"Birch Cliff, Cliffside West",1,1,1,1,0,0
"Brockton, Parkdale Village, Exhibition Place",1,1,1,1,0,0
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1,0,0


In [66]:
neighborhoodsSortedByVeg = neighborhoodsWithoutYogaWithVeg_Group.sort_values(by=['VegRest'], ascending=False)

In [67]:
neighborhoodsSortedByVeg.head()

,PostalCode,Borough,Latitude,Longitude,Venue,VegRest
Neighbourhood,,,,,,
"Kensington Market, Chinatown, Grange Park",4,4,4,4,0,4
"Richmond, Adelaide, King",3,3,3,3,0,3
"Toronto Dominion Centre, Design Exchange",1,1,1,1,0,1
"Harbourfront East, Union Station, Toronto Islands",1,1,1,1,0,1
"First Canadian Place, Underground city",1,1,1,1,0,1


# Results
### There are 2 neughbourhoods without Yoga Studio and 3-4 Vegetarian restaurantes; Kensington Market, Chinatown, Grange Park & Richmond, Adelaide, King for analysis
### We are going to study the characteristics of them by obtaining the most common venues in the neighbourhood.

In [92]:
onehot_toronto = pd.get_dummies(venues_toronto[['Venue Category']], prefix="", prefix_sep="")

onehot_toronto['Neighbourhood'] = venues_toronto['Neighbourhood'] 

fixed_columns = [onehot_toronto.columns[-1]] + list(onehot_toronto.columns[:-1])
onehot_toronto.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [93]:
toronto_grouped = onehot_toronto.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.0000

In [94]:
#Printing tthe 5 most common venues for the two Neighbourhoods:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4          Accessories Store  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1             Gym  0.12
2  Sandwich Place  0.12
3     Coffee Shop  0.12
4             Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2                Pizza Place  0.05
3  Middle Eastern Restaurant  0.05
4                   Pharmacy  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0       Italian Restaurant  0.12
1           Sandw

# Discussion
### Kensington Market, Chinatown, Grange Park, has many coffes and asian and vegearian restaurants as most common venues. People who enjoy this venues could potentially like Yoga.
### However, Richmond, Adelaide, King, has gyms (that could be competence for our Studio) and hotels that could mean that it is a tourist or business area.

# Conclusion
### The best place for our Yoga Studio is Kensington Market, Chinatown, Grange Park neighbourhood